In [ ]:
from datasets import load_dataset

dataset = load_dataset("nixiesearch/amazon-esci", split="train", streaming=True)


In [ ]:
print(dataset)

In [ ]:
! pip install -U sentence-transformers


In [ ]:
from torch.utils.data import Dataset
from sentence_transformers import InputExample
import random

class HuggingFaceTripletDataset(Dataset):
    def __init__(self, hf_dataset):
        self.hf_dataset = hf_dataset

    def __getitem__(self, idx):
        row = self.hf_dataset[idx]
        query = row["query"]
        positive_doc = row["doc"]
        
        # Randomly select one negative document from the list of negative documents
        negative_doc = random.choice(row["neg"])

        # Create an InputExample for the SentenceTransformer model
        return InputExample(texts=[query, positive_doc, negative_doc])

    def __len__(self):
        return 1000

# Wrap it in the custom dataset class
train_dataset = HuggingFaceTripletDataset(dataset)

train_dataset.__getitem__(0)


In [ ]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader
from datasets import Dataset
import torch

# Assuming you have your model and train_samples defined

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Define the loss function
train_loss = losses.TripletLoss(model)

# Define batch sizes
dataset_batch_size = 10000  # for creating datasets
train_batch_size = 32  # for training

# Function to process a batch of data
def process_batch(batch_texts, batch_labels):
    dataset = Dataset.from_dict({f"sentence_{idx}": text for idx, text in enumerate(zip(*batch_texts))})
    dataset = dataset.add_column("label", batch_labels)
    
    # Create a DataLoader for this batch
    batch_dataloader = DataLoader(dataset, shuffle=True, batch_size=train_batch_size)
    
    # Train on this batch
    model.fit(
        train_objectives=[(batch_dataloader, train_loss)],
        epochs=1,
        steps_per_epoch=len(batch_dataloader),
        show_progress_bar=True
    )

# Process data in batches
for i in range(0, len(train_samples), dataset_batch_size):
    batch_samples = train_samples[i:i+dataset_batch_size]
    
    # Separate texts and labels
    batch_texts = [sample.texts for sample in batch_samples]
    batch_labels = [sample.label for sample in batch_samples]
    
    print(f"Processing batch {i//dataset_batch_size + 1}")
    process_batch(batch_texts, batch_labels)

print("Training completed")

# Save the model
model.save('output/bi-encoder-model')

In [ ]:


# Wrap it in the custom dataset class
train_dataset = HuggingFaceTripletDataset(dataset)

# Now, this can be used with a dataloader for training


In [ ]:
train_dataset.__getitem__(0)


In [ ]:
from torch.utils.data import DataLoader
# Print one sample from the dataloader
# Define a custom collate function
def collate_fn(batch):
    return batch

# Use the custom collate function in the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)

# Print one sample from the dataloader
for batch in train_dataloader:
    print("Sample from dataloader:")
    for example in batch:
        print(example)
        print("---")
    break


In [ ]:
train_dataloader

In [ ]:
from sentence_transformers import SentenceTransformer, losses

# Initialize your SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Define the loss function, for example, TripletLoss
train_loss = losses.TripletLoss(model)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, steps_per_epoch=1, show_progress_bar=True)



In [6]:

# Ensure the model is trained on a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
# Check if CUDA is available
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

# If CUDA is not available, check for GPU details
if not cuda_available:
    print("CUDA is not available. Checking for GPU details...")
    
    # Check if any GPU is detected
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs detected: {gpu_count}")
    
    if gpu_count > 0:
        for i in range(gpu_count):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("No GPU detected.")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

CUDA Available: False
CUDA is not available. Checking for GPU details...
Number of GPUs detected: 0
No GPU detected.


In [4]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader, IterableDataset
import torch

# Ensure the model is trained on a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# torch.set_default_tensor_type('cuda' if torch.cuda.is_available() else 'cpu')

class TripletsDataset(IterableDataset):
    def __init__(self, dataset, chunk_size=1000):
        self.dataset = dataset
        self.chunk_size = chunk_size

    def __iter__(self):
        buffer = []
        for sample in self.dataset:
            query = sample["query"]
            positive = sample["doc"]
            negative = sample["neg"][0]

            buffer.append(InputExample(texts=[query, positive, negative]))
            
            if len(buffer) >= self.chunk_size:
                random.shuffle(buffer)
                yield from buffer
                buffer = []
            
        if buffer:
            yield from buffer

# Load the dataset with streaming
dataset = load_dataset("nixiesearch/amazon-esci", streaming=True)
train_dataset = dataset["train"]

# Create an IterableDataset
triplets_dataset = TripletsDataset(train_dataset, chunk_size=1000)

# Initialize the SentenceTransformer model and move it to the GPU if available
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model.to(device)

# Prepare the train dataloader
train_dataloader = DataLoader(triplets_dataset, batch_size=16)

# Define the triplet loss
train_loss = losses.TripletLoss(model=model)

# Train the model for one step
model.fit(
    [(train_dataloader, train_loss)],
    epochs=1,
    steps_per_epoch=10000,
    warmup_steps=100,
    show_progress_bar=True
)

print("Training completed for one step.")

# Optional: Save the model
model.save('output/bi-encoder-model')

cpu


KeyboardInterrupt: 

In [5]:
from datasets import load_dataset

dataset = load_dataset("nixiesearch/amazon-esci", split="train", streaming=True)
for example in dataset:
    print(example)
    break


In [6]:
next(iter(dataset))

{'query': 'kids probtiotic fiber',
 'doc': 'Fiber Prebiotic Sugar-Free Gummies with Inulin, Digestive Support for Kids & Adults - Apple Flavor (60 Ct) Chewable Gummies: Our fiber gummies for kids and adults contain 3.35g of Inulin, used to improve gut health, relieve constipation, reverse appetite loss and alleviate an upset stomach.\nFiber Supplements: The easy way to take a dose of pure superfood supplement, our fiber supplement gummies are made for both adults and kids and have a delicious apple flavor with no artificial colors, flavors, or additives.\nFiber for Kids & Adults; BeLive’s Prebiotic Kids Fiber Gummies include Chicory Root & Inulin which are highly effective fibers to improve digestive function and helps in achieving a healthy weight. For the best result, take it with BeLive’s Probiotic Gummies!\nFiber Gummies for Adults Sugar Free: Vegan Friendly, BeLive does not use gelatin or any animal products in the making of our fiber gummy.\nQuality Check at Every Stage; Blending

In [8]:

# Example: Prepare input for a triplet loss task
def collate_fn(batch):
    examples = []
    for item in batch:
        query, pos, neg = item['query'], item['positive'], item['negative']
        examples.append(InputExample(texts=[query, pos, neg]))
    return examples

In [10]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, batch_size=16, collate_fn=collate_fn)



In [11]:
from sentence_transformers import SentenceTransformer, losses

# Initialize your SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\SWAYATTA DAW\Documents\Projects\esci\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:

train_loss = losses.MultipleNegativesRankingLoss(model=model)


In [13]:
# Training loop
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100
)

AttributeError: 'dict' object has no attribute 'texts'

# Working

In [14]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader, IterableDataset
import random

# Load the dataset
dataset = load_dataset("nixiesearch/amazon-esci", split="train", streaming=True)

class TripletsDataset(IterableDataset):
    def __init__(self, dataset, chunk_size=1000):
        self.dataset = dataset
        self.chunk_size = chunk_size

    def __iter__(self):
        buffer = []
        for sample in self.dataset:
            query = sample["query"]
            positive = sample["doc"]
            negative = random.choice(sample["neg"])
            
            buffer.append(InputExample(texts=[query, positive, negative]))
            
            if len(buffer) >= self.chunk_size:
                random.shuffle(buffer)
                yield from buffer
                buffer = []
        
        if buffer:
            random.shuffle(buffer)
            yield from buffer

# Create an IterableDataset
triplets_dataset = TripletsDataset(dataset, chunk_size=1000)

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Prepare the train dataloader
train_dataloader = DataLoader(triplets_dataset, batch_size=32)

# Define the triplet loss
train_loss = losses.TripletLoss(model=model)

# Train the model
model.fit(
    [(train_dataloader, train_loss)],
    epochs=1,
    steps_per_epoch=10000,  # Adjust this based on your needs
    warmup_steps=100,
    show_progress_bar=True
)

print("Training completed.")

# Save the model
model.save('output/bi-encoder-model')

max_steps is given, it will override any value given in num_train_epochs
Parameter 'column'=['_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1', '_dataset_1

{'loss': 4.9415, 'grad_norm': 1.6116958856582642, 'learning_rate': 1.9191919191919194e-05, 'epoch': 0.09}


 10%|█         | 1000/10000 [5:59:50<53:32:02, 21.41s/it]

{'loss': 4.9238, 'grad_norm': 2.242837905883789, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.18}


 15%|█▌        | 1500/10000 [8:57:49<50:20:30, 21.32s/it]

...

{'loss': 4.8812, 'grad_norm': 3.167172908782959, 'learning_rate': 1.3131313131313132e-05, 'epoch': 0.62}


 40%|████      | 4000/10000 [23:47:54<35:12:50, 21.13s/it]

{'loss': 4.8748, 'grad_norm': 3.5477869510650635, 'learning_rate': 1.2121212121212122e-05, 'epoch': 0.7}


 42%|████▏     | 4183/10000 [24:52:26<34:58:58, 21.65s/it]

KeyboardInterrupt: 